In [5]:
from typing import List, Literal, Tuple, Union

Grade = Union[int, str]
WeightMethod = Literal["even", "proximity"]

def select_and_weight_adapters(
    target_grade: Grade,
    window_size: int,
    weight_method: WeightMethod = "even",
) -> Tuple[List[int], List[float]]:
    """
    Sliding-window selection and weighting for models numbered 2..12 (inclusive).

    Args:
        target_grade: center of the window; accepts int 2..12 or zero-padded str like "02".
        window_size: how many grades to include on each side of the target (>= 0).
        weight_method:
            - "even": every selected adapter weight == 1, by definition.
            - "proximity": weights decrease with absolute distance to target and
              are normalized so the average weight across the selection equals 1.

    Returns:
        (selected_grades, weights), where:
            - selected_grades is an ascending list of ints within [2, 12], clipped at bounds.
            - weights has the same length as selected_grades.

    Behavior notes:
        - The window always centers on target, then clips at 2 and 12 as needed.
        - For "proximity", raw weights use an inverse-distance profile: 1 / (1 + |g - target|).
          We then scale so that mean(weight) == 1 (i.e., sum == len(selection)).
    """
    # Parse and validate the target
    if isinstance(target_grade, str):
        if not target_grade.isdigit():
            raise ValueError("target_grade must be an int 2..12 or a zero-padded numeric string like '02'.")
        target = int(target_grade)
    else:
        target = int(target_grade)

    if not (2 <= target <= 12):
        raise ValueError("target_grade must be in [2, 12].")
    if window_size < 0:
        raise ValueError("window_size must be >= 0.")

    # Compute clipped window
    lo = max(2, target - window_size)
    hi = min(12, target + window_size)
    selected = list(range(lo, hi + 1))

    # Weights
    n = len(selected)
    if weight_method == "even":
        weights = [1.0] * n
    elif weight_method == "proximity":
        # Inverse-distance raw weights, target gets highest value
        raw = [1.0 / (1.0 + abs(g - target)) for g in selected]
        s = sum(raw)
        if s == 0:
            # Fallback, though this cannot happen with the formula above
            weights = [1.0] * n
        else:
            scale = n / s  # ensures average == 1
            weights = [round(w * scale, 2) for w in raw]
    else:
        raise ValueError("weight_method must be 'even' or 'proximity'.")

    return selected, weights


# -------------------------
# Quick sanity checks
# -------------------------
if __name__ == "__main__":
    # Example 1: target 6, window 2
    sel, w = select_and_weight_adapters(6, 2, "even")
    print("Ex1 selected:", sel)        # [4, 5, 6, 7, 8]
    print("Ex1 weights:", w)           # [1, 1, 1, 1, 1]

    # Example 2: target 2, window 3, proximity
    sel, w = select_and_weight_adapters(2, 3, "proximity")
    print("Ex2 selected:", sel)        # [2, 3, 4, 5]
    print("Ex2 weights:", w)           # element 0 (grade 2) is highest; mean == 1
    print("Ex2 mean weight:", sum(w)/len(w))

    # Example 3: target "10", window 1, proximity
    sel, w = select_and_weight_adapters("10", 1, "proximity")
    print("Ex3 selected:", sel)        # [9, 10, 11]
    print("Ex3 weights:", w)           # element 1 (grade 10) is highest; mean == 1
    print("Ex3 mean weight:", sum(w)/len(w))

    # Example 4: target 10, window 5, proximity
    sel, w = select_and_weight_adapters(10, 5, "proximity")
    print("Ex4 selected:", sel)        # [5, 6, 7, 8, 9, 10, 11, 12]
    print("Ex4 weights:", w)           # element 5 (grade 10) is highest; mean == 1
    print("Ex4 mean weight:", sum(w)/len(w))

Ex1 selected: [4, 5, 6, 7, 8]
Ex1 weights: [1.0, 1.0, 1.0, 1.0, 1.0]
Ex2 selected: [2, 3, 4, 5]
Ex2 weights: [1.92, 0.96, 0.64, 0.48]
Ex2 mean weight: 1.0
Ex3 selected: [9, 10, 11]
Ex3 weights: [0.75, 1.5, 0.75]
Ex3 mean weight: 1.0
Ex4 selected: [5, 6, 7, 8, 9, 10, 11, 12]
Ex4 weights: [0.41, 0.49, 0.61, 0.81, 1.22, 2.44, 1.22, 0.81]
Ex4 mean weight: 1.00125
